# LLM Iteration 24

**Aim**

To fix the implementation of using None for random seeds in the model.

**Prompt**

This does not work when trying to use no random seeds. Seeds should be None for each patient type in the dictionary seeds. Fix this bug. Return the updated Scenario class.

**Note**

Modifications have been made to the output from the LLM, indicated by:
```python
########## MODIFICATION ##########
change
##################################
```
This is either used to add in code from previous iterations if only certain sections were updated, used to correct the LLM's code if required, or used to check the model is working as expected.

**Additional code**

In [1]:
def trace(msg):
    '''
    Utility function for printing simulation.
    Set the TRACE constant to FALSE to turn tracing off.
    
    Params:
    -------
    msg: str
        string to print to screen.
    '''
    if TRACE:
        print(msg)

**LLM Output**

To address the issue where using None for random seeds does not work as intended, the Scenario class needs to be updated to correctly handle setting seeds as None for each patient type in the dictionary. Here is the revised Scenario class with the bug fixed:

In [2]:
class Scenario:
    def __init__(self, num_beds, run_length=60*2, seed=None):
        self.num_beds = num_beds
        self.run_length = run_length
        self.seed = seed
        self.default_interarrival_times = {
            "Stroke": 1.2,
            "TIA": 9.3,
            "Complex Neurological": 3.6,
            "Other": 3.2
        }
        self.random_seeds = {
            "Stroke": 12345,
            "TIA": 54321,
            "Complex Neurological": 98765,
            "Other": 67890
        }

    def create_distributions(self, num_runs):
        interarrival_distributions = {}
        seeds = self.generate_seeds(num_runs)
        for patient_type, seed in seeds.items():
            ### MODIFICATION - added [0] to seed as list not int ###
            if seed[0] is not None:
                np.random.seed(seed)
                interarrival_distributions[patient_type] = np.random.\
                    default_rng(seed)
            else:
                interarrival_distributions[patient_type] = np.random.\
                    default_rng()
        return interarrival_distributions

    def generate_seeds(self, num_runs):
        seeds = {}
        for patient_type, seed in self.random_seeds.items():
            if self.seed is not None:
                ### MODIFICATION - fixed error of not generating a list ###
                # seeds[patient_type] = seed + self.seed
                seeds[patient_type] = [seed + i + self.seed for i in \
                                       range(num_runs)]
            else:
                ### MODIFICATION - fixed error of not generating a list ###
                # seeds[patient_type] = None
                seeds[patient_type] = [None for i in range(num_runs)]
        return seeds

In this updated Scenario class:
- The generate_seeds method now correctly sets the seed as None for each patient type when the global seed is specified as None.
- When a global seed is provided, it is added to the base seed for each patient type. If no global seed is specified (None), then the seed for each patient type is set to None.
- This modification ensures that random seeds are handled appropriately when using None for all sampling distributions in the model.

**Code from previous iters**

In [3]:
import simpy
import numpy as np
import math
import pandas as pd

In [4]:
class StrokeWard:
    def __init__(self, scenario):
        self.env = simpy.Environment()
        self.beds = simpy.Resource(self.env, capacity=scenario.num_beds)
        self.patient_count = 0
        self.wait_times = []
        self.waiting_patients = []
        self.length_of_stay = {
            "Stroke": {"mean": 7.4, "std_dev": 8.6},
            "TIA": {"mean": 1.8, "std_dev": 2.3},
            "Complex Neurological": {"mean": 4.0, "std_dev": 5.0},
            "Other": {"mean": 3.8, "std_dev": 5.2}
        }
        
        self.interarrival_distributions = scenario.\
            create_distributions(num_runs=1)
        
        self.route_probabilities = {
            "Stroke": {"ESD": 0.13, "Rehab Unit": 0.24, "Other": 0.63},
            "TIA": {"ESD": 0.01, "Rehab Unit": 0.01, "Other": 0.98},
            "Complex Neurological": {"ESD": 0.05, "Rehab Unit": 0.11,
                                     "Other": 0.84},
            "Other": {"ESD": 0.10, "Rehab Unit": 0.05, "Other": 0.85}
        }

        ########## MODIFICATION ##########
        self.patients_in_beds = 0
        self.bed_in_use_store = []
        self.los_store = []
        self.patient_type_store = []
        ##################################

    def update_length_of_stay(self, patient_type, destination):
        if patient_type == 'Stroke':
            if destination == 'ESD':
                return {"mean": 4.6, "std_dev": 4.8}
            elif destination == 'Other':
                # Mortality rate for Stroke patients going to Other
                if np.random.rand() < 0.02:
                    # Adjusted parameters for mortality case
                    return {"mean": 7.0, "std_dev": 8.7}
        
        return self.length_of_stay[patient_type]

    def admit_patient(self, patient):
        arrival_time = round(self.env.now, 2)
        with self.beds.request() as bed:
            yield bed
            ########## MODIFICATION ##########
            bed_arrival_time = self.env.now
            self.patients_in_beds += 1
            self.bed_in_use_store.append(self.patients_in_beds)
            self.patient_type_store.append(patient.patient_type)
            ##################################

            wait_time = max(0, round(self.env.now - arrival_time, 2))
            self.wait_times.append(wait_time)
            self.waiting_patients.append(len(self.beds.queue))

            trace(f'Patient {patient.id} ({patient.patient_type}) admitted '
                  f'to the stroke ward at day {round(self.env.now, 2)}. '
                  f'Wait time: {wait_time} days')

            route_probabilities = list(self.route_probabilities[patient.\
                                       patient_type].values())
            route_names = list(self.route_probabilities[patient.\
                               patient_type].keys())
            
            rng = np.random.default_rng(patient.seed)
            route_choice = rng.choice(route_names, p=route_probabilities)

            length_of_stay_params = self.\
                update_length_of_stay(patient.patient_type, # destination
                                      route_choice)

            mu, sigma = calculate_mu_sigma(length_of_stay_params["mean"],
                                           length_of_stay_params["std_dev"])

            los = np.random.default_rng(patient.seed).lognormal(mu, sigma)
            yield self.env.timeout(round(los, 2))

            ########## MODIFICATION ##########
            self.patients_in_beds -= 1
            bed_leave_time = self.env.now
            self.los_store.append(round(bed_leave_time - bed_arrival_time, 2))
            ##################################

            trace(f'Patient {patient.id} ({patient.patient_type}) discharged '
                  f'from the stroke ward at day {round(self.env.now, 2)}')

            trace(f'Patient {patient.id} ({patient.patient_type}) routed to: '
                  f'{route_choice}')

    def patient_arrival(self, patient_type):
        while True:
            if patient_type in self.interarrival_distributions:
                interarrival_time = self.\
                    interarrival_distributions[patient_type].\
                    exponential(scenario.\
                                default_interarrival_times[patient_type])
                yield self.env.timeout(interarrival_time)
                patient = Patient(self.patient_count, patient_type,
                                  self.\
                                  interarrival_distributions[patient_type])
                self.patient_count += 1
                self.env.process(self.admit_patient(patient))

    def summary(self):
        mean_wait_time = round(np.mean(self.wait_times), 2)
        mean_waiting_patients = round(np.mean(self.waiting_patients), 2)
        
        summary_data = {
            'Mean Wait Time': [mean_wait_time],
            'Mean Waiting Patients': [mean_waiting_patients]
        }
        
        summary_df = pd.DataFrame(summary_data)

        return summary_df

In [5]:
class Patient:
    def __init__(self, id, patient_type, seed):
        self.id = id
        self.patient_type = patient_type
        self.seed = seed

def calculate_mu_sigma(mean, std_dev):
    v = std_dev ** 2
    m = mean
    phi = math.sqrt(v + m ** 2)
    mu = math.log((m ** 2) / phi)
    sigma = math.sqrt(math.log((phi ** 2) / (m ** 2)))
    return mu, sigma

In [6]:
def single_run(scenario):
    stroke_ward = StrokeWard(scenario)

    PATIENTS = ["Stroke", "TIA", "Complex Neurological", "Other"]
    for patient_type in PATIENTS:
        stroke_ward.env.process(stroke_ward.patient_arrival(patient_type))

    trace("Simulation started...")
    stroke_ward.env.run(until=scenario.run_length)
    trace("Simulation ended.")

    return stroke_ward

def multiple_replications(scenario, num_runs):
    results = []
    for i in range(num_runs):
        ### MODIFICATION - added seed=scenario.seed and scenario.run_length ###
        scenario_with_seeds = Scenario(scenario.num_beds,
                                       run_length=scenario.run_length,
                                       seed=scenario.seed)
        scenario_with_seeds.random_seeds = {k: v[i] for k, v in scenario.\
                                            generate_seeds(num_runs).items()}
        
        stroke_ward = single_run(scenario_with_seeds)
        result_summary = stroke_ward.summary()
        results.append(result_summary)

    df_results = pd.concat(results, keys=[f'Replication {i+1}' for i in \
                                          range(num_runs)])

    return df_results

In [7]:
TRACE = False

# Example usage with random seeds for each replication
scenario = Scenario(num_beds=10, seed=1)

num_replications = 5
final_results_random_seeds = multiple_replications(scenario, num_replications)

# Example usage with None instead of random seeds for all sampling 
# distributions
scenario_none_seeds = Scenario(num_beds=10, seed=None)
final_results_none_seeds = multiple_replications(scenario_none_seeds,
                                                 num_replications)

print(final_results_random_seeds)
print(final_results_none_seeds)

                 Mean Wait Time  Mean Waiting Patients
Replication 1 0            1.24                   1.66
Replication 2 0            0.80                   1.40
Replication 3 0            0.56                   0.62
Replication 4 0            0.06                   0.07
Replication 5 0            0.29                   0.37
                 Mean Wait Time  Mean Waiting Patients
Replication 1 0            0.20                   0.21
Replication 2 0            1.02                   1.56
Replication 3 0            0.05                   0.07
Replication 4 0            0.17                   0.22
Replication 5 0            0.88                   1.45


**Checks**

In [8]:
# Changing number of replications
scenario = Scenario(num_beds=10, seed=1)
num_replications = 10
results = multiple_replications(scenario, num_replications)

results

,,Mean Wait Time,Mean Waiting Patients
Replication 1,0,1.24,1.66
Replication 2,0,0.80,1.40
Replication 3,0,0.56,0.62
Replication 4,0,0.06,0.07
Replication 5,0,0.29,0.37
Replication 6,0,1.96,3.17
Replication 7,0,0.14,0.20
Replication 8,0,2.13,3.59
Replication 9,0,0.54,0.59
Replication 10,0,1.31,2.04


In [9]:
# Changing run length
scenario = Scenario(num_beds=10, run_length=1000, seed=1)
num_replications = 10
results = multiple_replications(scenario, num_replications)

results

,,Mean Wait Time,Mean Waiting Patients
Replication 1,0,1.26,1.88
Replication 2,0,4.32,6.67
Replication 3,0,1.58,2.30
Replication 4,0,7.34,11.50
Replication 5,0,1.25,1.89
Replication 6,0,4.83,7.64
Replication 7,0,2.36,3.51
Replication 8,0,0.79,1.22
Replication 9,0,0.90,1.33
Replication 10,0,1.26,1.87


In [10]:
# Changing number of beds
scenario = Scenario(num_beds=1, seed=1)
num_replications = 10
results = multiple_replications(scenario, num_replications)

results

,,Mean Wait Time,Mean Waiting Patients
Replication 1,0,44.55,73.30
Replication 2,0,42.08,62.96
Replication 3,0,51.20,75.40
Replication 4,0,38.64,55.38
Replication 5,0,63.35,90.38
Replication 6,0,57.29,86.57
Replication 7,0,43.08,67.40
Replication 8,0,61.45,102.38
Replication 9,0,51.15,83.55
Replication 10,0,51.08,74.89


In [14]:
# Checking implementation of random seeds
scenario = Scenario(num_beds=10, seed=1)
seeds = scenario.generate_seeds(num_runs=5)
print('With random seeds:')
for pat, seed in seeds.items():
    print(f'{pat}: seeds => {seed}')

scenario = Scenario(num_beds=10)
seeds = scenario.generate_seeds(num_runs=5)
print('\nWithout random seeds:')
for pat, seed in seeds.items():
    print(f'{pat}: seeds => {seed}')

With random seeds:
Stroke: seeds => [12346, 12347, 12348, 12349, 12350]
TIA: seeds => [54322, 54323, 54324, 54325, 54326]
Complex Neurological: seeds => [98766, 98767, 98768, 98769, 98770]
Other: seeds => [67891, 67892, 67893, 67894, 67895]

Without random seeds:
Stroke: seeds => [None, None, None, None, None]
TIA: seeds => [None, None, None, None, None]
Complex Neurological: seeds => [None, None, None, None, None]
Other: seeds => [None, None, None, None, None]


**Reflection**

The LLM attempted to fix the bug but it produced an error when ran so manual correction was used to fix this. The LLM forgot that it set up the generate_seeds seeds dictionary to hold a list, so this was changed. This change also required a modification of seeds.items() in create_distributions to ensure it was checking an integer not a list. There were some edits made to multiple_replications in order to be able to change the run length and random seeds used, as it initialises a new Scenario class in each replication so the run_length and seeds options were always the default before the change. Following these changes, the replication results work as expected, with random seeds being able to be used to control the output or having no seeds for a random output. Changing the number of beds changes the waiting times and number of patients waiting as expected and the run length of the model can be changed.